https://kernc.github.io/backtesting.py/doc/examples/Strategies%20Library.html
https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html

In [119]:
import pandas as pd
import os

# import backtesting and yfinance

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import yfinance as yf

# define backtesting strategy...simple SMA crossover
# see https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html

# more strategies avalable!!!
# https://kernc.github.io/backtesting.py/doc/examples/Strategies%20Library.html

class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()
            
            
# generate dataset compatible with backtesting strategy

def get_data(tickerSymbol, startdate='2010-1-1', enddate='2020-7-7'):
    # tickerSymbol = "NVTA"
    tickerData = yf.Ticker(tickerSymbol)
    tdf = tickerData.history(start=startdate, end=enddate).drop(['Dividends', 'Stock Splits'], axis=1)
    tdf.index.name = None
    return tdf

# run the backtest and plot the results

def backtest_results(ticker, start, end, quiet=False):
    tdf = get_data(ticker,startdate=start, enddate=end)
    bt = Backtest(tdf, SmaCross, cash=10000, commission=.002)
    output = bt.run()
    if not quiet:
        print(output)
        bt.plot()
    return output

## Get some of the top 'momentum' opportunitues

#### `xdays = days since the 50 sma crossed the 250 sma`

> should probably change these windows to be smaller

#### rev_spread is `(high revenue - low revenue)/average revenue` from yfinance data


In [66]:
momentum_stats = "/Users/mingay/coding/aws_mingquant/stocks/momentum_stats.txt"
momdf = pd.read_csv(momentum_stats, sep="\t")
momdf.columns = ['stock', 'xdays', 'rev_spread']
subdf = momdf[(momdf['xdays'] > 0) & (momdf['xdays'] < 20)].sort_values('rev_spread', ascending=False)
subdf.head()

,stock,xdays,rev_spread
190,UBER,4,0.624138
108,FND,18,0.347752
167,RAMP,4,0.188826
115,AMCR,6,0.139171
27,SFIX,4,0.117945


In [102]:
print([i for i in subdf.stock])

['UBER', 'FND', 'RAMP', 'AMCR', 'SFIX', 'NVTA', 'ORCL', 'UPWK', 'KDP', 'PANW', 'PS']


In [131]:
symbola = pd.read_csv("data/large_cap_volume_leaders.csv").Symbol
symbolb = pd.read_csv("data/nasdaq_volume_leaders.csv").Symbol
symbolc = pd.read_csv("data/mid_cap_volume_leaders.csv").Symbol
symbold = pd.read_csv("data/small_cap_volume.csv").Symbol
symbols = [i for i in set(symbola.append(symbolb).append(symbolc).append(symbold))]
symbols[:5]
# print(len(symbols))

['PLUG', 'NKE', 'CHWY', 'HPP', 'ALK']

In [133]:
masterdf = pd.DataFrame()
startdate='2019-07-01'
enddate='2020-07-07'
# stocks = [i for i in subdf.stock]
for ticker in symbols:
    try:
        results = backtest_results(ticker, startdate, enddate, quiet=True)
        rdf = pd.DataFrame(results).T
        rdf['advantage'] = rdf['Return [%]']/rdf['Buy & Hold Return [%]']
        rdf['symbol'] = ticker
        masterdf = pd.concat([masterdf.copy(), rdf])
    except:
        print("failed for: "+ ticker)
masterdf.to_csv("data/backtest_all_results.txt", sep="\t", index=False)
masterdf.head()

- PBR.A: No data found, symbol may be delisted
failed for: PBR.A
- DOWNLOADED FROM BARCHART.COM AS OF 05-25-2020 05:42PM CDT: No data found, symbol may be delisted
failed for: Downloaded from Barchart.com as of 05-25-2020 05:42pm CDT
- RDS.A: No data found, symbol may be delisted
failed for: RDS.A
- RDS.B: No data found for this date range, symbol may be delisted
failed for: RDS.B
- DOWNLOADED FROM BARCHART.COM AS OF 05-25-2020 05:32PM CDT: No data found, symbol may be delisted
failed for: Downloaded from Barchart.com as of 05-25-2020 05:32pm CDT
- BRK.B: No data found, symbol may be delisted
failed for: BRK.B
failed for: BPY
- DOWNLOADED FROM BARCHART.COM AS OF 05-25-2020 08:22PM CDT: No data found, symbol may be delisted
failed for: Downloaded from Barchart.com as of 05-25-2020 08:22pm CDT
- DOWNLOADED FROM BARCHART.COM AS OF 05-25-2020 11:23PM CDT: No data found, symbol may be delisted
failed for: Downloaded from Barchart.com as of 05-25-2020 11:23pm CDT
failed for: SO


,Start,End,Duration,Exposure [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Max. Drawdown [%],Avg. Drawdown [%],...,Max. Trade Duration,Avg. Trade Duration,Expectancy [%],SQN,Sharpe Ratio,Sortino Ratio,Calmar Ratio,_strategy,advantage,symbol
0,2019-07-01,2020-07-06,371 days,76.0108,9063.42,13213,-9.36585,345.536,-74.1389,-25.6569,...,40 days 00:00:00,24 days 00:00:00,14.8575,-1.33296,-0.30387,-0.731142,-0.0682762,SmaCross,-0.0271053,PLUG
0,2019-07-01,2020-07-06,371 days,80.0539,9404.06,11205.4,-5.95943,18.256,-31.3725,-7.58248,...,75 days 00:00:00,33 days 00:00:00,7.07916,-0.109762,0.00627075,0.00926261,0.00166964,SmaCross,-0.326436,NKE
0,2019-07-01,2020-07-06,371 days,88.6792,5288.04,10377.4,-47.1196,39.584,-57.9019,-29.5398,...,57 days 00:00:00,26 days 00:00:00,10.8728,-1.56955,-0.298495,-0.619234,-0.0634638,SmaCross,-1.19037,CHWY
0,2019-07-01,2020-07-06,371 days,86.7925,7437.9,12516.1,-25.621,21.5509,-40.8939,-13.7337,...,44 days 00:00:00,23 days 00:00:00,5.14196,-0.877614,-0.210171,-0.316041,-0.0415013,SmaCross,-1.18886,HPP
0,2019-07-01,2020-07-06,371 days,86.5229,4660.55,12586,-53.3945,40.8329,-65.3962,-20.4648,...,49 days 00:00:00,27 days 00:00:00,13.3296,-0.843612,-0.207516,-0.265889,-0.068665,SmaCross,-1.30763,ALK


In [141]:
scols = ['symbol', 'advantage', 'Return [%]', 'Buy & Hold Return [%]', '# Trades']
sdf = masterdf[scols][masterdf['advantage'] > 2]
sdf.sort_values('Return [%]', ascending=False).head()

,symbol,advantage,Return [%],Buy & Hold Return [%],# Trades
0,HOME,79.5624,651.782,8.19209,4
0,CLSN,4.91601,589.362,119.886,6
0,MTDR,7.34071,432.557,58.9258,7
0,NGL,5.62799,399.972,71.0684,8
0,CWH,3.01662,394.582,130.803,8


In [145]:
masterdf[scols].sort_values('Return [%]', ascending=True).head()

,symbol,advantage,Return [%],Buy & Hold Return [%],# Trades
0,DRAD,-1.72051,-100,58.1222,8
0,VIR,-0.519259,-100,192.582,5
0,ALT,-0.315232,-100,317.227,8
0,ACB,-1.14894,-100,87.037,10
0,TRIL,-0.0467675,-100,2138.24,5


In [142]:
print(len(sdf))

127
